# BEGES des entreprises

## Importation et doc Excel

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# import streamlit as st
# import streamlit.components.v1 as components

import altair as alt
from altair import expr, datum
import itertools 

In [3]:
file_name =  "./data/bilans-ges.xls"

df_assess = pd.read_excel(io=file_name, sheet_name="assessments")
df_emissions = pd.read_excel(io=file_name, sheet_name="Détail émissions")
df_postes = pd.read_excel(io=file_name, sheet_name="Postes")
df_scopes = pd.read_excel(io=file_name, sheet_name="Scopes")
df_descriptions = pd.read_excel(io=file_name, sheet_name="Descriptions")
df_emission_bilan = pd.read_excel(io=file_name, sheet_name="assessment_bilan")
df_reduction_bilan = pd.read_excel(io=file_name, sheet_name="assessment_reduction")

In [4]:
#PROCESS
# dico scope et poste
dico_postes = {x.id:x.label for (idx, x) in df_postes.iterrows()}
dico_scopes = {x.Scope:x.Description for (idx, x) in df_scopes.iterrows()}
dico_postes_to_scopes = {x.id:x.scope_id for (idx, x) in df_postes.iterrows()}

# process generally the data
#DF EMISSIONS
df_emissions = df_emissions.rename(columns={'scope_item_id':'poste_item_id'})
df_emissions[['total', 'co2_biogenic']] = df_emissions[['total', 'co2_biogenic']].fillna(0)
df_emissions['poste_item'] = df_emissions.poste_item_id.map(dico_postes)
df_emissions['scope_item'] = df_emissions.poste_item_id.map(dico_postes_to_scopes)

#DF REDUCTION BILAN

df_reduction_bilan[['reductions_scope_1_2', 'reductions_scope_1', 'reductions_scope_2',
       'reductions_scope_3']] = df_reduction_bilan[['reductions_scope_1_2', 'reductions_scope_1', 'reductions_scope_2',
       'reductions_scope_3']].fillna(0)

In [9]:
#INFO TO FILL
selected_organization_name = "Casino, Guichard Perrachon"

#GENERAL INFO 
loc_info = df_assess[df_assess.organization_name == selected_organization_name]
loc_assess_id = loc_info.id.values[0]
loc_reporting_year = df_emission_bilan[df_emission_bilan.assessment_id == loc_assess_id].reporting_year.values[0]
print(f'Reported year: {loc_reporting_year}')

#DESCRIPTION
loc_description  = df_descriptions[df_descriptions.assessment_id == loc_assess_id].copy()
loc_description_potentiel = loc_description.key.tolist()
selected_descrip = loc_description_potentiel[0] #TO SELECT BY THE USER IN THEORY

loc_descrip_to_print = loc_description[loc_description.key == selected_descrip].value.values[0]
print(selected_descrip)
print(loc_descrip_to_print)

#EMISSION
loc_emission = df_emissions[df_emissions.assessment_id == loc_assess_id]

loc_chart_poste_emission = alt.Chart(loc_emission).mark_bar().encode(
        x='poste_item_id',
        y='total:Q',
        color='scope_item:N',
        tooltip=[
            alt.Tooltip('poste_item', title='scope_item'),
        ]
        ).properties(
                width=500,
                height=300
                )
loc_total_emissions = loc_emission.total.sum()


#REDUCTION
loc_reduc_bilan = df_reduction_bilan[df_reduction_bilan.id == loc_assess_id].copy()
loc_has_plan = loc_reduc_bilan.action_plan.values[0] == 'Oui'
if loc_has_plan:
    print('Has a plan')
else:
    print('Has NOT a plan')

loc_reduc_bilan = loc_reduc_bilan[['id', 'reductions_scope_1_2', 'reductions_scope_1', 'reductions_scope_2',
       'reductions_scope_3']].melt(id_vars='id')

loc_reduced_emission = loc_reduc_bilan.value.sum()
print(f'Ratio of reduced emission: {loc_reduced_emission/loc_total_emissions:.2f} %')

loc_chart_reduced_emission = alt.Chart(loc_reduc_bilan).mark_bar().encode(
        x=alt.X('variable:N'),
        y='value:Q',
        ).properties(
                width=500,
                height=300
                )

Reported year: 2015
Plan d'action Scope 1
 
								
					Le volume de réduction attendu pour le scope 1 est de 99999.9 tCO2e					        		   
							 
 Priorité 1 : (Eviter) Amélioration du confinement des circuits frigorifiques existants. 
 Priorité 2 : (Réduire) Réduction de la charge initiale en HFC sur toutes les installations neuves 
 Priorité 3 : (Eviter) Mise en service d'installations transcritiques et de groupes au propane. 
Has a plan
Ratio of reduced emission: 0.22 %


In [13]:
loc_info#.organization_description.values[0]

,id,organization_name,organization_description,organization_type,collectivity_type,staff,population,consolidation_method,source_url
721,1286,"Casino, Guichard Perrachon",Grande distribution (magasins + entrepots + se...,Entreprise,NaN,74805.0,NaN,Opérationnel,http://www.bilans-ges.ademe.fr/fr/bilanenligne...


In [148]:
loc_reduc_bilan[['id', 'reductions_scope_1_2', 'reductions_scope_1', 'reductions_scope_2',
       'reductions_scope_3']].melt(id_vars='id')

,id,variable,value
0,15,reductions_scope_1_2,NaN
1,15,reductions_scope_1,NaN
2,15,reductions_scope_2,NaN
3,15,reductions_scope_3,NaN


In [133]:
np.isnan(loc_reduc_bilan.reductions_scope_1_2.values[0])

True

In [134]:
loc_reduc_bilan

,id,reporting_year,reference_year,action_plan,reductions_scope_1_2,reductions_scope_1,reductions_scope_2,reductions_scope_3
14,15,2011,NaN,Non,NaN,NaN,NaN,NaN


In [128]:
df_reduction_bilan.action_plan.unique()#[df_reduction_bilan.id == loc_assess_id]

array(['Oui', 'Non'], dtype=object)

In [90]:
loc_emission

,assessment_id,type,scope_item_id,co2,ch4,n2o,other,total,co2_biogenic,scope_item
55956,7071,Déclaration,1,2621.0,0.0,0.0,0.0,2621.0,0.0,Emissions directes des sources fixes de combus...
55957,7071,Déclaration,2,2639.0,0.0,0.0,0.0,2639.0,0.0,Emissions directes des sources mobiles à moteu...
55958,7071,Déclaration,3,0.0,0.0,0.0,0.0,0.0,0.0,Emissions directes des procédés hors énergie
55959,7071,Déclaration,4,4.0,0.0,0.0,0.0,4.0,0.0,Emissions directes fugitives
55960,7071,Déclaration,5,0.0,0.0,0.0,0.0,0.0,0.0,Emissions issues de la biomasse (sols et forêts)
55961,7071,Déclaration,6,NaN,NaN,NaN,NaN,44.0,0.0,Emissions indirectes liées à la consommation d...
55962,7071,Déclaration,7,NaN,NaN,NaN,NaN,0.0,0.0,Emissions indirectes liées à la consommation d...


In [89]:
alt.Chart(loc_emission).mark_bar().encode(
        x='scope_item_id',
        y='total:Q',
        tooltip=[
            alt.Tooltip('scope_item', title='scope_item'),
        ]
        ).properties(
                width=500,
                height=300
                )

alt.Chart(...)